In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as mplt
import matplotlib as plt
import datetime as dati

In [2]:
dataset = pd.read_csv("./data/friofrio/Train_TP2_Datos_2020-2C.csv", 
            parse_dates = ['Account_Created_Date', 'Opportunity_Created_Date',
                           'Planned_Delivery_Start_Date', 'Planned_Delivery_End_Date',
                           'Quote_Expiry_Date', 'Last_Modified_Date','Last_Activity' ])
# Limpio registros con muchos datos faltantes o inutiles
dataset = dataset.drop(columns= ['Prod_Category_A','Product_Category_B','Actual_Delivery_Date','Submitted_for_Approval','Last_Activity','Price','Currency','Brand','Size','Product_Type'])

dataset2 = pd.read_csv("./data/friofrio/Test_TP2_Datos_2020-2C.csv", 
                       parse_dates = ['Account_Created_Date', 'Opportunity_Created_Date',
                                      'Planned_Delivery_Start_Date', 'Planned_Delivery_End_Date',
                                      'Quote_Expiry_Date', 'Last_Modified_Date','Last_Activity' ])
# Limpio registros con muchos datos faltantes o inutiles
#dataset = dataset.drop(columns= [Brand','Size','Product_Type'])
dataset2 = dataset2.drop(columns= ['Prod_Category_A','Product_Category_B','Actual_Delivery_Date','Submitted_for_Approval','Last_Activity','Price','Currency','Brand','Size','Product_Type'])

In [3]:
#dataset.columns.intersection(dataset2.columns)
#dataset2.columns.intersection(['Region', 'Territory','Bureaucratic_Code', 'Billing_Country',
#                              'Account_Name','Account_Owner', 'Opportunity_Owner','Quote_Type', 'Delivery_Terms',
#               'Last_Modified_By', 'Product_Family','Product_Name', 'Month', 'Delivery_Quarter'])

dataset2.dtypes

ID                                             int64
Region                                        object
Territory                                     object
Pricing, Delivery_Terms_Quote_Appr             int64
Pricing, Delivery_Terms_Approved               int64
Bureaucratic_Code_0_Approval                   int64
Bureaucratic_Code_0_Approved                   int64
Bureaucratic_Code                             object
Account_Created_Date                  datetime64[ns]
Source                                        object
Billing_Country                               object
Account_Name                                  object
Opportunity_Name                              object
Opportunity_ID                                 int64
Sales_Contract_No                             object
Account_Owner                                 object
Opportunity_Owner                             object
Account_Type                                  object
Opportunity_Type                              

In [4]:
#limpio los datos no finalzizados
dataset = dataset.loc[(dataset['Stage']== 'Closed Won') | (dataset['Stage']== 'Closed Lost')]
dataset['Stage'].value_counts()

Closed Won     9533
Closed Lost    7350
Name: Stage, dtype: int64

In [5]:
#mergeo ambos DS con countes y ganados, en el caso de dataset 2 los cunters y los ganados son siempre 0 así no afectan los promedios.
dataset['ganado']= (dataset['Stage']== 'Closed Won' )
dataset.drop(columns= ['Stage'])
dataset['counter']= 1

dataset2['ganado']= 0
dataset2['counter']= 0

dataset_merge= dataset.append(dataset2).sort_values('Opportunity_Created_Date').reset_index().drop(columns=['index'])
#dataset_merge['counter'].value_counts()
dataset_merge.head()

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Bureaucratic_Code,Account_Created_Date,Source,...,Delivery_Quarter,Delivery_Year,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,ganado,counter
0,12714,EMEA,France,0,0,0,0,Bureaucratic_Code_4,2013-07-27,None,...,Q4,2018,2,EUR,662287.5,EUR,662287.5,Closed Lost,0,1
1,18684,APAC,Philippines,1,1,1,1,Bureaucratic_Code_3,2014-01-22,Source_3,...,Q2,2016,20,USD,9803430.0,USD,9803430.0,Closed Lost,0,1
2,20675,Americas,NW America,1,0,1,0,Bureaucratic_Code_5,2013-08-22,None,...,Q3,2016,0,USD,170931.6,USD,341863.2,Closed Lost,0,1
3,20682,Americas,NW America,1,0,1,0,Bureaucratic_Code_5,2013-08-22,None,...,Q3,2016,0,USD,170931.6,USD,341863.2,Closed Lost,0,1
4,20513,Americas,None,1,0,1,0,Bureaucratic_Code_5,2013-08-22,Source_7,...,Q1,2016,6,USD,4395304.0,USD,31410780.0,Closed Won,1,1


In [6]:
categorical = ['Region', 'Territory', 'Product_Name', 'Account_Owner','Bureaucratic_Code','Billing_Country', 
               'Account_Name', 'Opportunity_Owner','Quote_Type', 'Delivery_Terms', 'Product_Family', 
               'Month', 'Delivery_Quarter']

columnas_generadas_generadas = ['Opportunity_ID']

for agrupador in categorical:
    
    #probar con alguna de las dos lineas cual es mejor
    gruped_product = dataset_merge.groupby(agrupador)[['ganado','counter']].cumsum()
    #gruped_product = dataset_merge.groupby(agrupador)[['ganado','counter']].shift().cumsum()
    gruped_product[agrupador + '_mean']= np.NaN
    gruped_product.loc[(gruped_product['counter'] != 0), agrupador + '_mean'] = gruped_product.loc[(gruped_product['counter'] != 0), 'ganado']/ gruped_product.loc[(gruped_product['counter'] != 0),'counter']
    gruped_product= gruped_product.rename(columns = {'ganado':(agrupador + '_ganados'), 'counter': (agrupador + '_count') })
    dataset_merge= dataset_merge.join(gruped_product)
    #print(gruped_product.shape)
    columnas_generadas_generadas.append(agrupador + '_mean')
    #columnas_generadas_generadas.append(agrupador + '_sum')
    columnas_generadas_generadas.append(agrupador + '_count')


#agrupador = 'Opportunity_Owner' )
#gruped_product = dataset_merge.groupby(agrupador)[['ganado','counter']].cumsum()
#gruped_product[agrupador + '_mean']= np.NaN
#gruped_product.loc[(gruped_product['counter'] != 0), agrupador + '_mean'] = gruped_product.loc[(gruped_product['counter'] != 0), 'ganado']/ gruped_product.loc[(gruped_product['counter'] != 0),'counter']
#gruped_product= gruped_product.rename(columns = {'ganado':(agrupador + '_ganados'), 'counter': (agrupador + '_count') })

#dataset_merge= dataset_merge.join(gruped_product)
print(columnas_generadas_generadas)
gruped_product.head()

['Opportunity_ID', 'Region_mean', 'Region_count', 'Territory_mean', 'Territory_count', 'Product_Name_mean', 'Product_Name_count', 'Account_Owner_mean', 'Account_Owner_count', 'Bureaucratic_Code_mean', 'Bureaucratic_Code_count', 'Billing_Country_mean', 'Billing_Country_count', 'Account_Name_mean', 'Account_Name_count', 'Opportunity_Owner_mean', 'Opportunity_Owner_count', 'Quote_Type_mean', 'Quote_Type_count', 'Delivery_Terms_mean', 'Delivery_Terms_count', 'Product_Family_mean', 'Product_Family_count', 'Month_mean', 'Month_count', 'Delivery_Quarter_mean', 'Delivery_Quarter_count']


,Delivery_Quarter_ganados,Delivery_Quarter_count,Delivery_Quarter_mean
0,0,1,0.0
1,0,1,0.0
2,0,1,0.0
3,0,2,0.0
4,1,1,1.0


In [7]:
dataset_merge.head()

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Bureaucratic_Code,Account_Created_Date,Source,...,Delivery_Terms_mean,Product_Family_ganados,Product_Family_count,Product_Family_mean,Month_ganados,Month_count,Month_mean,Delivery_Quarter_ganados,Delivery_Quarter_count,Delivery_Quarter_mean
0,12714,EMEA,France,0,0,0,0,Bureaucratic_Code_4,2013-07-27,None,...,0.000000,0,1,0.0,0,1,0.0,0,1,0.0
1,18684,APAC,Philippines,1,1,1,1,Bureaucratic_Code_3,2014-01-22,Source_3,...,0.000000,0,1,0.0,0,1,0.0,0,1,0.0
2,20675,Americas,NW America,1,0,1,0,Bureaucratic_Code_5,2013-08-22,None,...,0.000000,0,1,0.0,0,1,0.0,0,1,0.0
3,20682,Americas,NW America,1,0,1,0,Bureaucratic_Code_5,2013-08-22,None,...,0.000000,0,2,0.0,0,1,0.0,0,2,0.0
4,20513,Americas,None,1,0,1,0,Bureaucratic_Code_5,2013-08-22,Source_7,...,0.333333,1,1,1.0,1,1,1.0,1,1,1.0


In [8]:
dataset.dtypes


ID                                             int64
Region                                        object
Territory                                     object
Pricing, Delivery_Terms_Quote_Appr             int64
Pricing, Delivery_Terms_Approved               int64
Bureaucratic_Code_0_Approval                   int64
Bureaucratic_Code_0_Approved                   int64
Bureaucratic_Code                             object
Account_Created_Date                  datetime64[ns]
Source                                        object
Billing_Country                               object
Account_Name                                  object
Opportunity_Name                              object
Opportunity_ID                                 int64
Sales_Contract_No                             object
Account_Owner                                 object
Opportunity_Owner                             object
Account_Type                                  object
Opportunity_Type                              

In [9]:
datetime_object = dati.datetime.strptime('1 1 2000', '%d %m %Y')
print(datetime_object)

2000-01-01 00:00:00


# categorical de a pares

In [10]:
#for i in range(len(categorical) -1):
#    for j in range(i +1, len(categorical)):
#        gruped_product = dataset_merge.groupby([categorical[i],categorical[j]])[['ganado','counter']].cumsum()
#        gruped_product[categorical[i] + '_'+ categorical[j] + '_mean']= np.NaN
#        gruped_product.loc[(gruped_product['counter'] != 0), categorical[i] + '_'+ categorical[j] + '_mean'] = gruped_product.loc[(gruped_product['counter'] != 0), 'ganado']/ gruped_product.loc[(gruped_product['counter'] != 0),'counter']
#        gruped_product[categorical[i] + '_'+ categorical[j] + '_mean'] = gruped_product['ganado']/ gruped_product['counter']
#        gruped_product= gruped_product.rename(columns = {'ganado':(categorical[i] + '_'+ categorical[j] + '_ganados'), 'counter': (categorical[i] + '_'+ categorical[j] + '_count') })
#        dataset_merge= dataset_merge.join(gruped_product)
        

In [11]:
dataset.dtypes

ID                                             int64
Region                                        object
Territory                                     object
Pricing, Delivery_Terms_Quote_Appr             int64
Pricing, Delivery_Terms_Approved               int64
Bureaucratic_Code_0_Approval                   int64
Bureaucratic_Code_0_Approved                   int64
Bureaucratic_Code                             object
Account_Created_Date                  datetime64[ns]
Source                                        object
Billing_Country                               object
Account_Name                                  object
Opportunity_Name                              object
Opportunity_ID                                 int64
Sales_Contract_No                             object
Account_Owner                                 object
Opportunity_Owner                             object
Account_Type                                  object
Opportunity_Type                              

# Convertir mondeas

In [12]:
dataset_merge['conversion'] = dataset_merge['ASP_(converted)']/dataset_merge['ASP']

cotizacion = dataset_merge.groupby(['ASP_Currency', 'Opportunity_Created_Date']).agg({'conversion': 'mean'}).reset_index()
#print(dataset[dataset['Stage']== 'Closed Won']['conversion'].value_counts())
cotizacion = cotizacion.rename(columns = {'conversion': 'conversion_diaria'})
cotizacion

dataset_merge= dataset_merge.merge(cotizacion, on = ['ASP_Currency', 'Opportunity_Created_Date'], how = 'left')

dataset_merge

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Bureaucratic_Code,Account_Created_Date,Source,...,Product_Family_count,Product_Family_mean,Month_ganados,Month_count,Month_mean,Delivery_Quarter_ganados,Delivery_Quarter_count,Delivery_Quarter_mean,conversion,conversion_diaria
0,12714,EMEA,France,0,0,0,0,Bureaucratic_Code_4,2013-07-27,None,...,1,0.000000,0,1,0.000000,0,1,0.000000,1.131079,1.131079
1,18684,APAC,Philippines,1,1,1,1,Bureaucratic_Code_3,2014-01-22,Source_3,...,1,0.000000,0,1,0.000000,0,1,0.000000,1.000000,1.000000
2,20675,Americas,NW America,1,0,1,0,Bureaucratic_Code_5,2013-08-22,None,...,1,0.000000,0,1,0.000000,0,1,0.000000,1.000000,1.000000
3,20682,Americas,NW America,1,0,1,0,Bureaucratic_Code_5,2013-08-22,None,...,2,0.000000,0,1,0.000000,0,2,0.000000,1.000000,1.000000
4,20513,Americas,None,1,0,1,0,Bureaucratic_Code_5,2013-08-22,Source_7,...,1,1.000000,1,1,1.000000,1,1,1.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19429,27867,EMEA,Belgium,1,1,1,1,Bureaucratic_Code_4,2015-06-17,Source_7,...,5,0.000000,10,19,0.526316,2218,4031,0.550236,1.131100,1.131100
19430,11318,Japan,None,1,1,0,0,Bureaucratic_Code_4,2019-02-12,None,...,0,NaN,0,5,0.000000,2603,4134,0.629657,0.008990,0.008990
19431,5817,Japan,Japan,1,1,0,0,Bureaucratic_Code_4,2017-06-30,None,...,74,0.500000,10,19,0.526316,2218,4031,0.550236,0.008990,0.008990
19432,16339,EMEA,KSA,1,1,0,0,Bureaucratic_Code_4,2017-06-12,Source_3,...,385,0.680519,0,2,0.000000,2518,4423,0.569297,1.000000,1.000000


In [13]:
dataset_merge['conversion'] = dataset_merge.loc[:,'conversion'].fillna(dataset_merge['conversion_diaria'])

In [14]:
dataset_merge = dataset_merge.drop(dataset_merge[dataset_merge['conversion'] == np.inf].index)

In [15]:
dataset_merge.loc[dataset_merge['ASP_Currency'] == 'USD', 'conversion'] = 1
dataset_merge.loc[dataset_merge.ASP_Currency == 'JPY' , 'conversion']= dataset_merge.loc[dataset_merge.ASP_Currency == 'JPY' , 'conversion'].fillna( dataset_merge.loc[dataset_merge.ASP_Currency == 'JPY', 'conversion'].mean())
dataset_merge.loc[dataset_merge.ASP_Currency == 'EUR' , 'conversion']= dataset_merge.loc[dataset_merge.ASP_Currency == 'EUR' , 'conversion'].fillna( dataset_merge.loc[dataset_merge.ASP_Currency == 'EUR', 'conversion'].mean())
dataset_merge['Total_Amount_converted'] = dataset_merge['Total_Amount'] * dataset_merge['conversion']

# diferencias de tiempo

In [16]:
dataset_merge['delta_planed'] = (dataset_merge['Planned_Delivery_End_Date'] - dataset_merge['Planned_Delivery_Start_Date']).dt.days
dataset_merge['delta_planed']

0         0.0
1        31.0
2        30.0
3        30.0
4        28.0
         ... 
19429    18.0
19430     0.0
19431     0.0
19432     9.0
19433    53.0
Name: delta_planed, Length: 19433, dtype: float64

In [17]:
dataset_merge.dtypes

ID                                      int64
Region                                 object
Territory                              object
Pricing, Delivery_Terms_Quote_Appr      int64
Pricing, Delivery_Terms_Approved        int64
                                       ...   
Delivery_Quarter_mean                 float64
conversion                            float64
conversion_diaria                     float64
Total_Amount_converted                float64
delta_planed                          float64
Length: 87, dtype: object

In [18]:
dataset_merge['acount_age'] = (dataset_merge['Opportunity_Created_Date'] - dataset_merge['Account_Created_Date']).dt.days

In [19]:
dataset_merge

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Bureaucratic_Code,Account_Created_Date,Source,...,Month_count,Month_mean,Delivery_Quarter_ganados,Delivery_Quarter_count,Delivery_Quarter_mean,conversion,conversion_diaria,Total_Amount_converted,delta_planed,acount_age
0,12714,EMEA,France,0,0,0,0,Bureaucratic_Code_4,2013-07-27,None,...,1,0.000000,0,1,0.000000,1.131079,1.131079,7.490997e+05,0.0,101
1,18684,APAC,Philippines,1,1,1,1,Bureaucratic_Code_3,2014-01-22,Source_3,...,1,0.000000,0,1,0.000000,1.000000,1.000000,9.803430e+06,31.0,58
2,20675,Americas,NW America,1,0,1,0,Bureaucratic_Code_5,2013-08-22,None,...,1,0.000000,0,1,0.000000,1.000000,1.000000,1.709316e+05,30.0,218
3,20682,Americas,NW America,1,0,1,0,Bureaucratic_Code_5,2013-08-22,None,...,1,0.000000,0,2,0.000000,1.000000,1.000000,1.709316e+05,30.0,218
4,20513,Americas,None,1,0,1,0,Bureaucratic_Code_5,2013-08-22,Source_7,...,1,1.000000,1,1,1.000000,1.000000,1.000000,4.395304e+06,28.0,264
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19429,27867,EMEA,Belgium,1,1,1,1,Bureaucratic_Code_4,2015-06-17,Source_7,...,19,0.526316,2218,4031,0.550236,1.131100,1.131100,2.570900e+05,18.0,1409
19430,11318,Japan,None,1,1,0,0,Bureaucratic_Code_4,2019-02-12,None,...,5,0.000000,2603,4134,0.629657,0.008990,0.008990,1.823450e+05,0.0,73
19431,5817,Japan,Japan,1,1,0,0,Bureaucratic_Code_4,2017-06-30,None,...,19,0.526316,2218,4031,0.550236,0.008990,0.008990,4.557082e+04,0.0,665
19432,16339,EMEA,KSA,1,1,0,0,Bureaucratic_Code_4,2017-06-12,Source_3,...,2,0.000000,2518,4423,0.569297,1.000000,1.000000,2.301000e+04,9.0,683


# features por oportunity ID

In [20]:
#Opportunity_ID
features= ['sum','mean','median','std', 'max', 'min']

dataset_numerico = dataset_merge[['Opportunity_ID','ASP', 'ASP_(converted)', 'TRF', 'Total_Amount', 'Total_Taxable_Amount', 'Total_Amount_converted']]
#ASP                                           float64
#ASP_(converted)                               float64
#TRF                                             int64
#Total_Amount                                  float64
#Total_Taxable_Amount                          float64
#Total_Amount_converted                        float64

In [21]:
dataset_numerico
features_num= dataset_numerico.groupby('Opportunity_ID').agg(features)

features_num
level_0 = features_num.columns.get_level_values(0)
level_1 = features_num.columns.get_level_values(1)
features_num.columns = level_0 + '_' + level_1
features_num

,ASP_sum,ASP_mean,ASP_median,ASP_std,ASP_max,ASP_min,ASP_(converted)_sum,ASP_(converted)_mean,ASP_(converted)_median,ASP_(converted)_std,...,Total_Taxable_Amount_median,Total_Taxable_Amount_std,Total_Taxable_Amount_max,Total_Taxable_Amount_min,Total_Amount_converted_sum,Total_Amount_converted_mean,Total_Amount_converted_median,Total_Amount_converted_std,Total_Amount_converted_max,Total_Amount_converted_min
Opportunity_ID,,,,,,,,,,,,,,,,,,,,,
0,0.5200,0.5200,0.5200,NaN,0.5200,0.5200,0.58817,0.58817,0.58817,NaN,...,5272800.0,NaN,5272800.0,5272800.0,5.964044e+06,5.964044e+06,5.964044e+06,NaN,5.964044e+06,5.964044e+06
1,0.5300,0.5300,0.5300,NaN,0.5300,0.5300,0.59948,0.59948,0.59948,NaN,...,48230.0,NaN,48230.0,48230.0,5.455268e+04,5.455268e+04,5.455268e+04,NaN,5.455268e+04,5.455268e+04
2,0.4800,0.4800,0.4800,NaN,0.4800,0.4800,0.48000,0.48000,0.48000,NaN,...,83865.6,NaN,83865.6,83865.6,8.386560e+04,8.386560e+04,8.386560e+04,NaN,8.386560e+04,8.386560e+04
3,0.5300,0.5300,0.5300,NaN,0.5300,0.5300,0.53000,0.53000,0.53000,NaN,...,7421881.5,NaN,7421881.5,7421881.5,7.421882e+06,7.421882e+06,7.421882e+06,NaN,7.421882e+06,7.421882e+06
4,0.5300,0.5300,0.5300,NaN,0.5300,0.5300,0.53000,0.53000,0.53000,NaN,...,13357192.5,NaN,13357192.5,13357192.5,1.335719e+07,1.335719e+07,1.335719e+07,NaN,1.335719e+07,1.335719e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12799,0.5150,0.5150,0.5150,NaN,0.5150,0.5150,0.58251,0.58251,0.58251,NaN,...,401700.0,NaN,401700.0,401700.0,4.543578e+05,4.543578e+05,4.543578e+05,NaN,4.543578e+05,4.543578e+05
12800,1.0600,0.5300,0.5300,0.0,0.5300,0.5300,1.06000,0.53000,0.53000,0.0,...,21332500.0,0.0,21332500.0,21332500.0,2.133250e+07,1.066625e+07,1.066625e+07,120674.843277,1.075158e+07,1.058092e+07
12801,1.5600,0.5200,0.5200,0.0,0.5200,0.5200,1.76451,0.58817,0.58817,0.0,...,299715.0,0.0,299715.0,299715.0,3.390065e+05,1.130022e+05,1.168988e+05,6749.154643,1.168988e+05,1.052089e+05


In [22]:
dataset_merge = dataset_merge.merge(features_num, on= 'Opportunity_ID', how = 'left')

# binarios y tiempos por oportunities

In [23]:
dataset_merge.dtypes

ID                                      int64
Region                                 object
Territory                              object
Pricing, Delivery_Terms_Quote_Appr      int64
Pricing, Delivery_Terms_Approved        int64
                                       ...   
Total_Amount_converted_mean           float64
Total_Amount_converted_median         float64
Total_Amount_converted_std            float64
Total_Amount_converted_max            float64
Total_Amount_converted_min            float64
Length: 124, dtype: object

In [25]:
dataset_para_mean_dates_bin= dataset_merge.loc[ : ,['Opportunity_ID','Pricing, Delivery_Terms_Quote_Appr','Pricing, Delivery_Terms_Approved', 'Bureaucratic_Code_0_Approval', 'Bureaucratic_Code_0_Approved', 'Account_Created_Date', 'Last_Modified_Date', 'Opportunity_Created_Date','Quote_Expiry_Date','Planned_Delivery_Start_Date','Planned_Delivery_End_Date']]

In [26]:
dataset_para_mean_dates_bin[['Account_Created_Date', 'Last_Modified_Date', 'Opportunity_Created_Date','Quote_Expiry_Date','Planned_Delivery_Start_Date','Planned_Delivery_End_Date']] = (dataset_para_mean_dates_bin[['Account_Created_Date', 'Last_Modified_Date', 'Opportunity_Created_Date','Quote_Expiry_Date','Planned_Delivery_Start_Date','Planned_Delivery_End_Date']] - datetime_object)
dateFields = ['Account_Created_Date', 'Last_Modified_Date', 'Opportunity_Created_Date','Quote_Expiry_Date','Planned_Delivery_Start_Date','Planned_Delivery_End_Date']

for field in dateFields:
    dataset_para_mean_dates_bin[field] = dataset_para_mean_dates_bin[field].dt.days
                                                 
dataset_para_mean_dates_bin

#dataset_para_mean_dates_bin=dataset_para_mean_dates_bin.groupby('Opportunity_ID').agg('mean')

,Opportunity_ID,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Account_Created_Date,Last_Modified_Date,Opportunity_Created_Date,Quote_Expiry_Date,Planned_Delivery_Start_Date,Planned_Delivery_End_Date
0,12369,0,0,0,0,4956,6832,5057,6855.0,6869,6869.0
1,12370,1,1,1,1,5135,6052,5193,5956.0,5987,6018.0
2,12371,1,0,1,0,4982,6108,5200,NaN,6057,6087.0
3,12371,1,0,1,0,4982,6108,5200,NaN,6026,6056.0
4,12372,1,0,1,0,4982,5731,5246,5659.0,5844,5872.0
...,...,...,...,...,...,...,...,...,...,...,...
19428,12363,1,1,1,1,5646,7055,7055,7083.0,7121,7139.0
19429,12362,1,1,0,0,6982,7055,7055,NaN,7213,7213.0
19430,12361,1,1,0,0,6390,7055,7055,NaN,7121,7121.0
19431,12364,1,1,0,0,6372,7055,7055,7083.0,7060,7069.0


In [27]:
features_num = features_num.merge(dataset_para_mean_dates_bin, on= 'Opportunity_ID')

features_num.head()

,Opportunity_ID,ASP_sum,ASP_mean,ASP_median,ASP_std,ASP_max,ASP_min,ASP_(converted)_sum,ASP_(converted)_mean,ASP_(converted)_median,...,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Account_Created_Date,Last_Modified_Date,Opportunity_Created_Date,Quote_Expiry_Date,Planned_Delivery_Start_Date,Planned_Delivery_End_Date
0,0,0.52,0.52,0.52,NaN,0.52,0.52,0.58817,0.58817,0.58817,...,1,1,1,1,5645,6008,5819,5873.0,5965,6025.0
1,1,0.53,0.53,0.53,NaN,0.53,0.53,0.59948,0.59948,0.59948,...,0,0,0,0,5645,5858,5819,5863.0,5861,5863.0
2,2,0.48,0.48,0.48,NaN,0.48,0.48,0.48000,0.48000,0.48000,...,0,0,0,0,5589,6116,5820,6117.0,5868,5868.0
3,3,0.53,0.53,0.53,NaN,0.53,0.53,0.53000,0.53000,0.53000,...,1,0,1,0,4956,6660,5820,NaN,6606,6664.0
4,4,0.53,0.53,0.53,NaN,0.53,0.53,0.53000,0.53000,0.53000,...,1,0,1,0,4956,6660,5820,6362.0,6606,6633.0


In [28]:
features_num

,Opportunity_ID,ASP_sum,ASP_mean,ASP_median,ASP_std,ASP_max,ASP_min,ASP_(converted)_sum,ASP_(converted)_mean,ASP_(converted)_median,...,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Account_Created_Date,Last_Modified_Date,Opportunity_Created_Date,Quote_Expiry_Date,Planned_Delivery_Start_Date,Planned_Delivery_End_Date
0,0,0.5200,0.5200,0.5200,NaN,0.5200,0.5200,0.58817,0.58817,0.58817,...,1,1,1,1,5645,6008,5819,5873.0,5965,6025.0
1,1,0.5300,0.5300,0.5300,NaN,0.5300,0.5300,0.59948,0.59948,0.59948,...,0,0,0,0,5645,5858,5819,5863.0,5861,5863.0
2,2,0.4800,0.4800,0.4800,NaN,0.4800,0.4800,0.48000,0.48000,0.48000,...,0,0,0,0,5589,6116,5820,6117.0,5868,5868.0
3,3,0.5300,0.5300,0.5300,NaN,0.5300,0.5300,0.53000,0.53000,0.53000,...,1,0,1,0,4956,6660,5820,NaN,6606,6664.0
4,4,0.5300,0.5300,0.5300,NaN,0.5300,0.5300,0.53000,0.53000,0.53000,...,1,0,1,0,4956,6660,5820,6362.0,6606,6633.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19428,12801,1.5600,0.5200,0.5200,0.0,0.5200,0.5200,1.76451,0.58817,0.58817,...,1,1,1,1,5858,5862,5816,NaN,5931,5934.0
19429,12801,1.5600,0.5200,0.5200,0.0,0.5200,0.5200,1.76451,0.58817,0.58817,...,1,1,1,1,5858,5862,5816,NaN,5938,5942.0
19430,12801,1.5600,0.5200,0.5200,0.0,0.5200,0.5200,1.76451,0.58817,0.58817,...,1,1,1,1,5858,5862,5816,NaN,5924,5928.0
19431,12802,0.6375,0.6375,0.6375,NaN,0.6375,0.6375,0.63750,0.63750,0.63750,...,1,1,1,1,5771,5865,5817,5903.0,5959,5963.0


# fetaures cumulativas agrupadas

In [29]:
features= ['mean','median', 'max', 'min']
dataset_mean_ecoding = dataset_merge[columnas_generadas_generadas]
features_num_mean= dataset_mean_ecoding.groupby('Opportunity_ID').agg(features)
level_0 = features_num_mean.columns.get_level_values(0)
level_1 = features_num_mean.columns.get_level_values(1)
features_num_mean.columns = level_0 + '_' + level_1
features_num_mean.reset_index()

#dataset_merge = dataset_merge.join(features_num, on= 'Opportunity_ID', how = 'left')

,Opportunity_ID,Region_mean_mean,Region_mean_median,Region_mean_max,Region_mean_min,Region_count_mean,Region_count_median,Region_count_max,Region_count_min,Territory_mean_mean,...,Month_count_max,Month_count_min,Delivery_Quarter_mean_mean,Delivery_Quarter_mean_median,Delivery_Quarter_mean_max,Delivery_Quarter_mean_min,Delivery_Quarter_count_mean,Delivery_Quarter_count_median,Delivery_Quarter_count_max,Delivery_Quarter_count_min
0,0,0.309091,0.309091,0.309091,0.309091,165.0,165.0,165,165,0.273171,...,47,47,0.385965,0.385965,0.385965,0.385965,171.000000,171.0,171,171
1,1,0.313253,0.313253,0.313253,0.313253,166.0,166.0,166,166,0.274939,...,99,99,0.288848,0.288848,0.288848,0.288848,547.000000,547.0,547,547
2,2,0.283951,0.283951,0.283951,0.283951,243.0,243.0,243,243,0.504348,...,100,100,0.288043,0.288043,0.288043,0.288043,552.000000,552.0,552,552
3,3,0.280992,0.280992,0.280992,0.280992,242.0,242.0,242,242,0.500000,...,4,4,0.286751,0.286751,0.286751,0.286751,551.000000,551.0,551,551
4,4,0.283333,0.283333,0.283333,0.283333,240.0,240.0,240,240,0.508929,...,3,3,0.287796,0.287796,0.287796,0.287796,549.000000,549.0,549,549
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11352,12799,0.292208,0.292208,0.292208,0.292208,154.0,154.0,154,154,1.000000,...,78,78,0.392857,0.392857,0.392857,0.392857,168.000000,168.0,168,168
11353,12800,0.289980,0.289980,0.290598,0.289362,234.5,234.5,235,234,0.078745,...,32,31,0.126330,0.126330,0.127660,0.125000,47.500000,47.5,48,47
11354,12801,0.282874,0.282895,0.287582,0.278146,152.0,152.0,153,151,0.638889,...,163,77,0.316060,0.280148,0.389222,0.278810,414.666667,538.0,539,167
11355,12802,0.286920,0.286920,0.286920,0.286920,237.0,237.0,237,237,0.076923,...,80,80,0.388235,0.388235,0.388235,0.388235,170.000000,170.0,170,170


In [30]:
features_num = features_num.merge(features_num_mean,on= 'Opportunity_ID')

In [31]:
features_num

,Opportunity_ID,ASP_sum,ASP_mean,ASP_median,ASP_std,ASP_max,ASP_min,ASP_(converted)_sum,ASP_(converted)_mean,ASP_(converted)_median,...,Month_count_max,Month_count_min,Delivery_Quarter_mean_mean,Delivery_Quarter_mean_median,Delivery_Quarter_mean_max,Delivery_Quarter_mean_min,Delivery_Quarter_count_mean,Delivery_Quarter_count_median,Delivery_Quarter_count_max,Delivery_Quarter_count_min
0,0,0.5200,0.5200,0.5200,NaN,0.5200,0.5200,0.58817,0.58817,0.58817,...,47,47,0.385965,0.385965,0.385965,0.385965,171.000000,171.0,171,171
1,1,0.5300,0.5300,0.5300,NaN,0.5300,0.5300,0.59948,0.59948,0.59948,...,99,99,0.288848,0.288848,0.288848,0.288848,547.000000,547.0,547,547
2,2,0.4800,0.4800,0.4800,NaN,0.4800,0.4800,0.48000,0.48000,0.48000,...,100,100,0.288043,0.288043,0.288043,0.288043,552.000000,552.0,552,552
3,3,0.5300,0.5300,0.5300,NaN,0.5300,0.5300,0.53000,0.53000,0.53000,...,4,4,0.286751,0.286751,0.286751,0.286751,551.000000,551.0,551,551
4,4,0.5300,0.5300,0.5300,NaN,0.5300,0.5300,0.53000,0.53000,0.53000,...,3,3,0.287796,0.287796,0.287796,0.287796,549.000000,549.0,549,549
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19428,12801,1.5600,0.5200,0.5200,0.0,0.5200,0.5200,1.76451,0.58817,0.58817,...,163,77,0.316060,0.280148,0.389222,0.278810,414.666667,538.0,539,167
19429,12801,1.5600,0.5200,0.5200,0.0,0.5200,0.5200,1.76451,0.58817,0.58817,...,163,77,0.316060,0.280148,0.389222,0.278810,414.666667,538.0,539,167
19430,12801,1.5600,0.5200,0.5200,0.0,0.5200,0.5200,1.76451,0.58817,0.58817,...,163,77,0.316060,0.280148,0.389222,0.278810,414.666667,538.0,539,167
19431,12802,0.6375,0.6375,0.6375,NaN,0.6375,0.6375,0.63750,0.63750,0.63750,...,80,80,0.388235,0.388235,0.388235,0.388235,170.000000,170.0,170,170


In [32]:
deltatime_features= dataset_merge[['acount_age','delta_planed','Opportunity_ID']].groupby('Opportunity_ID').agg('mean').reset_index()

deltatime_features
features_num = features_num.merge(deltatime_features, on= 'Opportunity_ID')

In [33]:
features_num_mean.head()

features_num = features_num.merge(features_num_mean,on= 'Opportunity_ID')

# elimino las columnas no propias

In [34]:
filter = dataset2['ID'].tolist()

In [35]:
dataset_merge.shape

(19433, 124)

In [36]:

dataset_merge = dataset_merge.drop(columns=['Region', 'Territory','Bureaucratic_Code','Account_Type', 'Billing_Country', 'Opportunity_Type',
                              'Account_Name','Account_Owner', 'Opportunity_Owner','Quote_Type', 'Delivery_Terms',
                            'Pricing, Delivery_Terms_Quote_Appr',
                            'Sales_Contract_No', 'Opportunity_ID', 'Opportunity_Name','Source ', 'Account_Created_Date',
                            'Bureaucratic_Code_0_Approval', 'Bureaucratic_Code_0_Approved','Pricing, Delivery_Terms_Approved',
               'Last_Modified_By', 'Product_Family','Product_Name', 'Month', 'Delivery_Quarter','ganado','counter',
                            'Total_Taxable_Amount_Currency','Total_Taxable_Amount','Total_Amount_Currency','Total_Amount',
                           'TRF','Delivery_Year','Planned_Delivery_Start_Date', 'Planned_Delivery_End_Date', 'ASP_(converted)_Currency',
                           'ASP_(converted)', 'ASP_Currency', 'ASP', 'Quote_Expiry_Date', 'Last_Modified_Date','Opportunity_Created_Date'])

dataset_merge.shape

(19433, 82)

In [37]:
Test = dataset_merge.loc[dataset_merge['ID'].isin(filter)]
print(Test.shape)
Train = dataset_merge.loc[~dataset_merge['ID'].isin(filter)]
print(Train.shape)          

(2551, 82)
(16882, 82)


In [38]:
Test.to_csv('Features_Test_Lucas.csv')
Train.to_csv('Features_Train_Lucas.csv')

In [39]:
filter = dataset2['Opportunity_ID'].tolist()

Test = features_num.loc[features_num['Opportunity_ID'].isin(filter)]
print(Test.shape)
Train = features_num.loc[~features_num['Opportunity_ID'].isin(filter)]
print(Train.shape)          




(2551, 257)
(16882, 257)


In [ ]:
Test.to_csv('Features_agrupados_Test_Lucas.csv')
Train.to_csv('Features_agrupados_Train_Lucas.csv')